<h1> Context-Aware Recommender Systems</h1>
![Recommendation Systems Approaches](https://raw.githubusercontent.com/ziababar/recommender/master/images/Recommendation%20Systems.jpg)
In content-based recommender systems, we use the context information (such as time, location etc.) while building recommendation engines. These recommender systems are next generation recommendations systems, which fall into the hyper-personalization category. Though content-based recommender systems are efficient, targeted at an individual level, and consider the user's personal preferences alone while building recommendation engines, people wanted recommendation engines to be more personalized.

![Context-Aware Recommender Steps](https://raw.githubusercontent.com/ziababar/recommender/master/images/context-aware-1.png)
A typical content-based recommender system will perform the following steps:
1. Retrieve user, item and activity data
1. Define Context
2. Create a context profile with respect to a user for item content.
3. Generate recommendations for a context
![Context-Aware Recommender Steps](https://raw.githubusercontent.com/ziababar/recommender/master/images/context-aware-2.png)

<h2>Step 1 - Retrieve Data</h2>
The first step would always be to gather the data and pull it into the programming environment.
For our use case, we download the MovieLens dataset containing three sets of data,

Movie data containing a certain movie's information, such as movieID, release date, URL, genre details, and so on
User data containing the user information, such as userID, age, gender, occupation, ZIP code, and so on
Ratings data containing userID, itemID, rating, timestamp

In [ ]:
# Import the libraries that are going to be used here
import pandas as pd
import numpy as np
import scipy
import sklearn
import datetime

In [ ]:
# Column headers for the dataset
data_cols = ['user id','movie id','rating','timestamp']
item_cols = ['movie id','movie title','release date', 'video release date','IMDb URL','unknown','Action', 'Adventure','Animation','Childrens','Comedy','Crime', 'Documentary','Drama','Fantasy','Film-Noir','Horror', 'Musical','Mystery','Romance ','Sci-Fi','Thriller', 'War' ,'Western']
user_cols = ['user id','age','gender','occupation', 'zip code']

In [ ]:
# List of users
df_u_user = pd.read_csv('/home/nbuser/library/dataset/u.user', header=None, sep='|', names=user_cols, encoding='latin-1')
df_u_user = df_u_user.sort_values('user id', ascending=1)
df_u_user.columns
df_u_user.head(10)

In [ ]:
# List of movie items
df_u_item = pd.read_csv('/home/nbuser/library/dataset/u.item', header=None, sep='|', names=item_cols, encoding='latin-1')
df_u_item = df_u_item.sort_values('movie id', ascending=1)
df_u_item.columns
df_u_item.head(10)

In [ ]:
# Remove the several columns from the dataframe as it's not required for this analysis
df_u_item_final = df_u_item.drop(['release date', 'video release date','IMDb URL'], axis=1)
df_u_item_final.head(10)

In [ ]:
# User activity data
df_u_data = pd.read_csv('/home/nbuser/library/dataset/u.data', header=None, sep='\t', names=data_cols, encoding='latin-1')
df_u_data = df_u_data.sort_values('user id', ascending=1)
df_u_data.columns
df_u_data.head(10)

In [ ]:
df_u_data.insert(4, 'hour', '0')
df_u_data.head()

<h2>Step 2 - Define context</h2>
The timestamp information available in the dataset is used as the context for calculating the preference value for movie genres for each user for each hour of the day. This context profile information is used for generating context aware recommendations.

In [ ]:
# Loop through the entire dataset and convert the timestamp into the hour for each record
for i in range(0, len(df_u_data.index)):
    d = datetime.datetime.fromtimestamp(df_u_data.loc[i, ['timestamp']])
    df_u_data.loc[i, ['hour']] = d.hour

In [ ]:
# timestamp column is not longer required, drop it
df_u_data_final = df_u_data.drop(['timestamp'], axis=1)
df_u_data_final.head()

In [ ]:
# Merge the Movies and Ratings datasets using merge() function
ratings = pd.merge(df_u_item, df_u_data_final, on='movie id')
ratings.head(5)

<h2>Step 3 - Create a context profile for user </h2>

Since hour-of-day is being used as the context here, our recommendations will be made as per the time of the day. The set of recommendations for an active user will be different for each hour of the day.

In [ ]:
# Pick a user for whom a context profile needs to be created. Sample user IDs are 10, 500, 900, 943
UCP_pref = ratings.loc[ratings['user id'] == 943, ['hour', 'unknown','Action', 'Adventure','Animation','Childrens','Comedy','Crime', 'Documentary','Drama','Fantasy','Film-Noir','Horror', 'Musical','Mystery','Romance ','Sci-Fi','Thriller', 'War' ,'Western']]
UCP_pref

The records need to be aggregated so that we just get records by the hour.

In [ ]:
#Use the groupby operator to sum all the ratings across the genres
UCP_pref_sc = UCP_pref.groupby(['hour']).sum()
UCP_pref_sc

In [ ]:
Normalize the above dataset so that is actually scaled from 0 to 1

In [ ]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(UCP_pref_sc)
df_normalized = pd.DataFrame(np_scaled)
df_normalized

<h2>Step 4 - Generating context-aware recommendations</h2>
Now that we have created the context profile for the active user, context-aware recommendations for the user can be generated.

The following matrixes should be available in order for this to be done,
 - user context profile (UCP_Pref_SC)
 - user content recommendations (UCP_Pref_content).

For generating recommendations for a user at the ninth hour, an element wise multiplication of user content recommendations and the context row for the ninth hour of the day from the UCP_pref_SC object needs to be performed.